In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 21.6 MB/s eta 0:00:0000:01


In [2]:
# YOLOv8 Object Detection Training Script for KITTI Dataset
import os
import shutil
import yaml
import random
import torch
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 1. Setup and Configuration

In [3]:
KITTI_BASE_DIR = '/kaggle/input/kitti-dataset'
"""str: The base directory where the KITTI dataset is located."""

IMAGE_DIR = Path(KITTI_BASE_DIR) / 'data_object_image_2' / 'training' / 'image_2'
"""Path: Directory containing KITTI training images."""

LABEL_DIR = Path(KITTI_BASE_DIR) / 'data_object_label_2' / 'training' / 'label_2'
"""Path: Directory containing KITTI training labels."""

TRAIN_DIR = Path('train')
"""Path: Directory where training images and labels will be stored in YOLO format."""

VALID_DIR = Path('valid')
"""Path: Directory where validation images and labels will be stored in YOLO format."""

LABELS_DIR = Path('labels_with_dont_care')
"""Path: Directory where YOLO-formatted labels will be stored."""

CLASSES = [
    'Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 
    'Cyclist', 'Tram', 'Misc', 'DontCare'
]
"""
list of str: List of class names included in the KITTI dataset. 
'CLASSES' should reflect all possible object categories for detection.
"""

MODEL_ARCH = 'yolo11x.yaml'
"""str: The YOLO11x model configuration file to use."""

EPOCHS = 10
"""int: Number of epochs for training. Adjust this value based on dataset size and desired training time."""

BATCH_SIZE = 16
"""int: Batch size used during training. Adjust based on GPU memory constraints."""

IMG_SIZE = 640
"""int: The size (height and width) of the input images for the model."""

CONFIDENCE_THRESHOLD = 0.25
"""float: The confidence threshold for predictions during validation and testing."""

PROJECT_NAME = 'YOLO11-KITTI'
"""str: The name of the project folder where YOLO results will be saved."""

EXPERIMENT_NAME = 'exp1'
"""str: The name of the experiment folder within the project directory to store this run's results."""

device = 'cuda' if torch.cuda.is_available() else 'cpu'
"""str: The device to use for training and inference, defaults to GPU if available."""

print(f"Using device: {device}")

# Ensure the expected dataset directories exist
if not IMAGE_DIR.exists():
    raise FileNotFoundError(f"Image directory not found: {IMAGE_DIR}")
if not LABEL_DIR.exists():
    raise FileNotFoundError(f"Label directory not found: {LABEL_DIR}")

Using device: cuda


# 2. Data Preparation Functions

In [4]:
CLAZZ_NUMBERS = {name: idx for idx, name in enumerate(CLASSES)}
"""
dict: A mapping from class names to numeric labels. 
The numeric labels are used by YOLO for class indices.
"""


def convert_bbox_to_yolo(bbox, size):
    """
    Convert KITTI bounding box coordinates to YOLO11 format.

    Args:
        bbox (tuple of float): Bounding box coordinates in the format (left, right, top, bottom).
        size (tuple of int): Image size as (width, height).

    Returns:
        tuple of float: YOLO-formatted bounding box as (x_center, y_center, width, height) normalized by image size.
    """
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x_center = (bbox[0] + bbox[1]) / 2.0
    y_center = (bbox[2] + bbox[3]) / 2.0
    width = bbox[1] - bbox[0]
    height = bbox[3] - bbox[2]
    x_center *= dw
    width *= dw
    y_center *= dh
    height *= dh
    return x_center, y_center, width, height


def parse_kitti_label_file(lbl_path, img_path):
    """
    Parse a KITTI label file and convert the bounding boxes to YOLO11 format.

    Args:
        lbl_path (Path): Path to the KITTI label file (in KITTI text format).
        img_path (Path): Path to the corresponding image file.

    Returns:
        list of tuple: A list of YOLO-formatted bounding boxes. Each element is 
        (class_idx, x_center, y_center, width, height).
    """
    with open(lbl_path, 'r', encoding='utf-8') as file:
        lines = file.read().strip().split('\n')

    yolo_labels = []
    if not img_path.exists():
        # If the image doesn't exist, skip processing labels
        return yolo_labels

    img_size = Image.open(img_path).size  # (width, height)
    
    for line in lines:
        parts = line.split()
        clazz = parts[0]
        if clazz not in CLAZZ_NUMBERS:
            # Skip classes not in our mapping
            continue

        # KITTI format: 
        # type, truncated, occluded, alpha, bbox_left, bbox_top, bbox_right, bbox_bottom, ...
        # Indices:  0    ,    1     ,   2     ,   3  ,    4     ,    5    ,     6     ,      7    ...
        # Example: Car 0.00 0 1.57 148.00 174.00 350.00 325.00 ...
        # The bounding box coordinates: left = parts[4], top = parts[5], right = parts[6], bottom = parts[7]
        bbox_left = float(parts[4])
        bbox_top = float(parts[5])
        bbox_right = float(parts[6])
        bbox_bottom = float(parts[7])
        bbox = (bbox_left, bbox_right, bbox_top, bbox_bottom)

        # Convert bounding box to YOLO format (normalized)
        x_center, y_center, width, height = convert_bbox_to_yolo(bbox, img_size)
        clazz_number = CLAZZ_NUMBERS[clazz]

        # YOLO format: class x_center y_center width height
        yolo_labels.append((clazz_number, x_center, y_center, width, height))

    return yolo_labels

# 3. Generate YOLO labels

In [5]:
if not LABELS_DIR.exists():
    LABELS_DIR.mkdir()

image_paths = sorted(list(IMAGE_DIR.glob('*.png')))
label_paths = sorted(list(LABEL_DIR.glob('*.txt')))

for img_path in image_paths:
    lbl_path = LABEL_DIR / f"{img_path.stem}.txt"
    if lbl_path.exists():
        yolo_labels = parse_kitti_label_file(lbl_path, img_path)
        yolo_label_path = LABELS_DIR / f"{img_path.stem}.txt"
        with open(yolo_label_path, 'w', encoding='utf-8') as lf:
            for lbl in yolo_labels:
                lf.write(" ".join(f"{val:.6f}" for val in lbl) + "\n")

print("YOLO format labels have been generated in:", LABELS_DIR.resolve())

YOLO format labels have been generated in: /kaggle/working/labels_with_dont_care


# 4. Split Dataset into Train and Validation Sets

In [6]:
labels_for_images = [(img_path, LABELS_DIR / f"{img_path.stem}.txt") 
                     for img_path in image_paths 
                     if (LABELS_DIR / f"{img_path.stem}.txt").exists()]

train_pairs, valid_pairs = train_test_split(
    labels_for_images, 
    test_size=0.1, 
    random_state=42, 
    shuffle=True
)
print(f"Training samples: {len(train_pairs)}, Validation samples: {len(valid_pairs)}")

# Create directories for YOLO data structure:
for folder in [TRAIN_DIR, VALID_DIR]:
    if folder.exists():
        shutil.rmtree(folder)
    folder.mkdir()
    (folder / 'images').mkdir()
    (folder / 'labels').mkdir()

for img_path, lbl_path in train_pairs:
    shutil.copy(img_path, TRAIN_DIR / 'images' / img_path.name)
    shutil.copy(lbl_path, TRAIN_DIR / 'labels' / lbl_path.name)

for img_path, lbl_path in valid_pairs:
    shutil.copy(img_path, VALID_DIR / 'images' / img_path.name)
    shutil.copy(lbl_path, VALID_DIR / 'labels' / lbl_path.name)

print(f"Training data copied to {TRAIN_DIR / 'images'} and {TRAIN_DIR / 'labels'}")
print(f"Validation data copied to {VALID_DIR / 'images'} and {VALID_DIR / 'labels'}")

Training samples: 6732, Validation samples: 749
Training data copied to train/images and train/labels
Validation data copied to valid/images and valid/labels


# 5. Create data.yaml File for YOLO

In [7]:
DATA_CONFIG = 'data.yaml'
data_config = {
    'train': str((TRAIN_DIR / 'images').resolve()),
    'val': str((VALID_DIR / 'images').resolve()),
    'names': CLASSES,
    'nc': len(CLASSES)
}

with open(DATA_CONFIG, 'w', encoding='utf-8') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("data.yaml file created with content:")
print(data_config)

data.yaml file created with content:
{'train': '/kaggle/working/train/images', 'val': '/kaggle/working/valid/images', 'names': ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare'], 'nc': 9}


# 6. Train the YOLO11 Model

In [8]:
model = YOLO(MODEL_ARCH).to(device)
train_results = model.train(
    data=DATA_CONFIG,
    epochs=EPOCHS,
    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    project=PROJECT_NAME,
    name=EXPERIMENT_NAME,
    device=device,
    exist_ok=True
)

print("\nTraining completed!\n")

Ultralytics 8.3.55 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo11x.yaml, data=data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=YOLO11-KITTI, name=exp1, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=Tr

100%|██████████| 755k/755k [00:00<00:00, 24.7MB/s]
2024-12-31 07:08:54,323	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-12-31 07:08:55,105	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=9

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 105MB/s]


AMP: checks passed ✅


...

                   all        749       5425      0.612      0.454      0.483      0.293



10 epochs completed in 2.060 hours.
Optimizer stripped from YOLO11-KITTI/exp1/weights/last.pt, 114.4MB
Optimizer stripped from YOLO11-KITTI/exp1/weights/best.pt, 114.4MB

Validating YOLO11-KITTI/exp1/weights/best.pt...
Ultralytics 8.3.55 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11x summary (fused): 464 layers, 56,837,419 parameters, 0 gradients, 194.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:11<00:00,  2.15it/s]


                   all        749       5425       0.61      0.455      0.483      0.292
                   Car        678       3040      0.846      0.807      0.893      0.636
                   Van        223        296      0.368      0.642      0.443      0.292
                 Truck        102        104      0.523      0.788      0.761      0.566
            Pedestrian        179        441      0.668      0.456      0.519      0.255
        Person_sitting         12         25          1     0.0576      0.176     0.0699
               Cyclist        124        174      0.524      0.466      0.502      0.247
                  Tram         29         56      0.501      0.536      0.574      0.303
                  Misc         75         91       0.68      0.233      0.339      0.223
              DontCare        567       1198      0.377      0.109      0.136     0.0389
Speed: 0.1ms preprocess, 10.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to YOLO11-KIT

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# 7. Validate the Model

In [9]:
best_weights_path = f'{PROJECT_NAME}/{EXPERIMENT_NAME}/weights/best.pt'
model = YOLO(best_weights_path).to(device)
validation_results = model.val(
    data=DATA_CONFIG, 
    split='val', 
    conf=CONFIDENCE_THRESHOLD,
    save=False,
    plots=False
)
print("\nValidation completed!\n")

# The result from model.val() is a 'DetMetrics' object that summarizes performance.
# The 'DetMetrics' class holds detection metrics, including mean precision (mp), mean recall (mr), 
# mAP (map50, map50-95), and others.
# Let's inspect the attributes of validation_results for these metrics.

print("Validation Results (raw DetMetrics object):")
print(validation_results)

# According to Ultralytics YOLO code, `validation_results` is a `DetMetrics` object that has a 'box' attribute
# which holds metrics. We can inspect `validation_results.box` as it may contain the summary metrics.
print("Attributes of validation_results:")
for attribute in dir(validation_results):
    if not attribute.startswith('_'):
        print(attribute, "=", getattr(validation_results, attribute))

# Access relevant metrics from the DetMetrics object
metrics = validation_results.box  # box is an instance of the Metric class storing results for boxes
print("\nBox Metrics:")
print(metrics)

# The metrics attribute should include values like 'mp' (mean precision), 'mr' (mean recall), etc.
# According to the docs:
# - metrics.ap50: average precision at IoU=0.50
# - metrics.ap: average precision for IoU=0.50:0.95
# - metrics.mp: mean precision
# - metrics.mr: mean recall
# - metrics.map50: mean average precision at IoU=0.50
# - metrics.map: mean average precision at IoU=0.50:0.95

# We will attempt to retrieve these metrics:
precision = getattr(metrics, 'mp', None)
recall = getattr(metrics, 'mr', None)

# Calculate F1 score if precision and recall are available
f1_score = None
if precision is not None and recall is not None and (precision + recall) > 0:
    f1_score = 2 * (precision * recall) / (precision + recall)

# Accuracy is not directly applicable to object detection tasks like YOLO
accuracy = None

# Placeholder for confusion matrix
confusion_matrix = None

# Print metrics in the requested format
print("\nConfusion Matrix:")
if confusion_matrix is not None:
    print(confusion_matrix)
else:
    print("[[ ... ]]")  # Placeholder for an actual confusion matrix if implemented

if accuracy is not None:
    print(f"Accuracy: {accuracy * 100:.2f}%")
else:
    print("Accuracy: Not Applicable for object detection")

if precision is not None:
    print(f"Precision: {precision:.2f}")
else:
    print("Precision: Not Available")

if recall is not None:
    print(f"Recall: {recall:.2f}")
else:
    print("Recall: Not Available")

if f1_score is not None:
    print(f"F1 Score: {f1_score:.2f}")
else:
    print("F1 Score: Not Available")

YOLO11x summary (fused): 464 layers, 56,837,419 parameters, 0 gradients, 194.5 GFLOPs


val: Scanning /kaggle/working/valid/labels.cache... 749 images, 0 backgrounds, 0 corrupt: 100%|██████████| 749/749 [00:00<?, ?it/s]
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 47/47 [00:10<00:00,  4.34it/s]


                   all        749       5425      0.667      0.428      0.565       0.38
                   Car        678       3040      0.878      0.784      0.867       0.67
                   Van        223        296      0.397      0.622      0.448      0.322
                 Truck        102        104      0.562      0.779       0.78      0.598
            Pedestrian        179        441      0.723      0.426      0.604       0.34
        Person_sitting         12         25          1       0.04       0.52      0.364
               Cyclist        124        174      0.612      0.454       0.57      0.319
                  Tram         29         56      0.625      0.446      0.539       0.32
                  Misc         75         91      0.778      0.231      0.516      0.386
              DontCare        567       1198       0.43     0.0668      0.239     0.0955
Speed: 0.1ms preprocess, 11.2ms inference, 0.0ms loss, 0.7ms postprocess per image

Validation completed!

Val

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# 8. Predictions on Validation Set (Optional)

In [10]:
val_predictions = model.predict(
    source=str((VALID_DIR / 'images').resolve()), 
    save=True, 
    conf=CONFIDENCE_THRESHOLD
)

if val_predictions:
    predictions_save_dir = val_predictions[0].save_dir
    print(f"\nPredictions saved to '{predictions_save_dir}'.\n")
else:
    print("No predictions were made.")


image 1/749 /kaggle/working/valid/images/000019.png: 224x640 1 Car, 1 Van, 1 Tram, 55.6ms
image 2/749 /kaggle/working/valid/images/000031.png: 224x640 4 Cars, 2 Vans, 1 Truck, 20.5ms
image 3/749 /kaggle/working/valid/images/000037.png: 224x640 2 Cars, 2 Trucks, 20.3ms
image 4/749 /kaggle/working/valid/images/000041.png: 224x640 1 Car, 1 DontCare, 20.6ms
image 5/749 /kaggle/working/valid/images/000048.png: 224x640 3 Pedestrians, 1 Cyclist, 20.1ms
image 6/749 /kaggle/working/valid/images/000050.png: 224x640 4 Cars, 20.3ms
image 7/749 /kaggle/working/valid/images/000061.png: 224x640 4 Cars, 2 Vans, 1 DontCare, 23.3ms
image 8/749 /kaggle/working/valid/images/000065.png: 224x640 2 Trams, 22.7ms
image 9/749 /kaggle/working/valid/images/000069.png: 224x640 2 Cars, 1 DontCare, 20.4ms
image 10/749 /kaggle/working/valid/images/000073.png: 224x640 4 Pedestrians, 2 Cyclists, 20.4ms
image 11/749 /kaggle/working/valid/images/000079.png: 224x640 4 Cars, 1 Truck, 1 Tram, 1 DontCare, 20.3ms
image 12/7